In [5]:
"""
Ball detection by dumbest method - per-pixel comparison by MatchTemplate
Target ball radius should be known.
No green filter needed.
"""
import cv2
import numpy as np;
from random import randrange
from math import pi, fabs, sin, cos
from numpy.linalg import inv

# Target ball radius
radius = 50.0

# Preparing input image
im_rgb = cv2.imread("../data/images/ball.png", cv2.IMREAD_COLOR)
im_gray = cv2.cvtColor(im_rgb, cv2.COLOR_BGR2GRAY)

# Drawing a synthetic ball image
template = np.zeros( ( int(radius*2.0*1.8), int(radius*2.0*1.8), 3), np.uint8)
template[:] = (0, 128, 0)
cv2.circle(template,(int(template.shape[0]/2), int(template.shape[1]/2)), int(radius), (255,255,255),-1)
cv2.imshow("template", template)
cv2.waitKey(0)


# Apply template Matching
w = template.shape[1]
h = template.shape[0]
res = cv2.matchTemplate(im_rgb, template, cv2.TM_CCOEFF)
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
top_left = max_loc
cv2.imshow("res", res/max_val)
cv2.waitKey(0)

# Drawing result
bottom_right = (top_left[0] + w, top_left[1] + h)
cv2.rectangle(im_rgb,top_left, bottom_right, 255, 2)
cv2.imshow("im_rgb", im_rgb)


cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
"""
Ball detection by HoughCircles
No green filter needed.
Known ball radius used to filter false positives.
"""
import cv2
import numpy as np;

# Target ball radius
radius = 50.0

# Preparing input image
im_rgb = cv2.imread("../data/images/ball.png", cv2.IMREAD_COLOR)
im = cv2.cvtColor(im_rgb, cv2.COLOR_BGR2GRAY)
im_verbose = cv2.cvtColor(im, cv2.COLOR_GRAY2BGR)

# Doing HoughCircles
circles = cv2.HoughCircles(im, cv2.HOUGH_GRADIENT, dp=2, minDist=30, param1=300, param2=80, \
                           minRadius=int(radius-10), maxRadius=int(radius+10) )

# Drawing result
circles = np.uint16(np.around(circles))
for i in circles[0,:]:
    cv2.circle(im_verbose,(i[0],i[1]),i[2],(0,0,255),4)

cv2.imshow('detected circles',im_verbose)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
"""
Ball detection by Canny -> FindContours -> Circularity check
No green filter needed
"""

import cv2
import numpy as np;
from random import randrange
from math import pi, fabs

# Preparing input image
im_rgb = cv2.imread("../data/images/ball.png", cv2.IMREAD_COLOR)
im_gray = cv2.cvtColor(im_rgb, cv2.COLOR_BGR2GRAY)

# Doing Canny
edges = cv2.Canny(im_gray, 100, 200)
cv2.imshow('edges',edges)
cv2.waitKey(0)

# Doing FindContours
contours, hierarchy = cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

# Checking contours circularity and drawing result
for contour in contours:
    color = (randrange(255), randrange(255), randrange(255))
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    if(perimeter>200):
        circularity = 4*pi*area/(perimeter*perimeter)
        if circularity > 0.5:
            #cv2.drawContours(im_rgb, contour, -1, color, 3)
            cv2.drawContours(im_rgb, contour, -1, (0,0,255), 3)
cv2.imshow('drawContours',im_rgb)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
"""
Ball detection by MSER -> ConvexHull -> Area ratio check 
"""

import cv2
import numpy as np;
from random import randrange
from math import pi, fabs


# Preparing input image
im_rgb = cv2.imread("../data/images/ball.png", cv2.IMREAD_COLOR)
im_gray = cv2.cvtColor(im_rgb, cv2.COLOR_BGR2GRAY)

# Doing MSER
mser = cv2.MSER_create()
regions, _ = mser.detectRegions(im_gray)
hulls = [cv2.convexHull(p.reshape(-1, 1, 2)) for p in regions]
cv2.polylines(im_rgb, hulls, 1, (0, 0, 255), 2)
cv2.imshow('im_rgb',im_rgb)
cv2.waitKey(0)

# Filtering results and picking best one
best_center = (-1,-1)
best_radius = int(0)
best_area_coeff = 0
for region in regions:
    color = (randrange(128,255), randrange(128,255), randrange(128,255))
    hull = np.int32([cv2.convexHull(region.reshape(-1, 1, 2))])
    cv2.polylines(im_rgb, hull, True, color, 1)
    cv2.polylines(im_rgb, np.int32([region]), True, color, 1)
    (x,y),radius = cv2.minEnclosingCircle(hull[0])
    center = (int(x),int(y))
    area_hull = cv2.contourArea(hull[0])
    area_circle = pi*radius*radius
    area_coeff = area_hull/area_circle
    if area_coeff > best_area_coeff:
        best_area_coeff = area_coeff
        best_center = center
        best_radius = int(radius)
if best_area_coeff > 0.9:
    cv2.circle(im_rgb, best_center, best_radius, (0,0,255),5)

cv2.imshow('mser',im_rgb)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
"""
Ball detection by adaptiveThreshold -> Skeletonize -> Ransac
"""

import cv2
import numpy as np;
from random import randrange
from math import pi, fabs, sin, cos
from numpy.linalg import inv

""" Simple RANSAC circle class refactored from https://github.com/SeongHyunBae/RANSAC-circle-python """
class SIMPLE_CIRCLE_RANSAC:    
	def __init__(self, x_data, y_data, n):
		self.x_data = x_data
		self.y_data = y_data
		self.n = n
		self.best_inliers = 0
		self.best_model = None

	def random_sampling(self):
		sample = []
		save_ran = []
		count = 0

		# get three points from data
		while True:
			ran = np.random.randint(len(self.x_data))

			if ran not in save_ran:
				sample.append((self.x_data[ran], self.y_data[ran]))
				save_ran.append(ran)
				count += 1

				if count == 3:
					break

		return sample

	def make_model(self, sample):
		# calculate A, B, C value from three points by using matrix

		pt1 = sample[0]
		pt2 = sample[1]
		pt3 = sample[2]

		A = np.array([[pt2[0] - pt1[0], pt2[1] - pt1[1]], [pt3[0] - pt2[0], pt3[1] - pt2[1]]])
		B = np.array([[pt2[0]**2 - pt1[0]**2 + pt2[1]**2 - pt1[1]**2], [pt3[0]**2 - pt2[0]**2 + pt3[1]**2 - pt2[1]**2]])

		inv_A = inv(A)

		c_x, c_y = np.dot(inv_A, B) / 2
		c_x, c_y = c_x[0], c_y[0]
		r = np.sqrt((c_x - pt1[0])**2 + (c_y - pt1[1])**2)

		return c_x, c_y, r

	def eval_model(self, model):
		inliers = 0
		c_x, c_y, r = model

		if (r > 15) and (r < 25): # hardcoded target radius
			for i in range(len(self.x_data)):
				dis = np.sqrt((self.x_data[i]-c_x)**2 + (self.y_data[i]-c_y)**2)
				if fabs(dis-r) < 3.0:  # hardcoded inliers distance tolerance
					inliers += 1

		return inliers

	def execute_ransac(self):
		for i in range(self.n):
			try:
				model = self.make_model(self.random_sampling())
			except np.linalg.LinAlgError as err:
				continue

			inliers = self.eval_model(model)

			if inliers > self.best_inliers:
				self.best_model = model
				self.best_inliers = inliers
		


im_rgb = cv2.imread("../data/images/ball.png", cv2.IMREAD_COLOR)
im_rgb = cv2.pyrDown(im_rgb)

im_gray = cv2.cvtColor(im_rgb, cv2.COLOR_BGR2GRAY)

im = cv2.adaptiveThreshold(im_gray, 128, cv2.ADAPTIVE_THRESH_MEAN_C,  cv2.THRESH_BINARY_INV, 41, 30)

im_verbose = cv2.cvtColor(im, cv2.COLOR_GRAY2BGR)
cv2.imshow("adaptiveThreshold", im_verbose)
cv2.waitKey(0)

size = np.size(im)
skel = np.zeros(im.shape,np.uint8)
ret,im = cv2.threshold(im,127,255,0)
 
if 0:
	# Doing morphological skeletonisation with barebone OpenCV
	element = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
	done = False
	while( not done):
		eroded = cv2.erode(im,element)
		temp = cv2.dilate(eroded,element)
		temp = cv2.subtract(im,temp)
		skel = cv2.bitwise_or(skel,temp)
		im = eroded.copy()
	
		zeros = size - cv2.countNonZero(im)
		if zeros==size:
			done = True
else:
	# Doing Zhang-Suen skeletonisation (gives better results, but requires opencv-contrib-python package
	skel = cv2.ximgproc.thinning(im)

cv2.imshow("skel", skel)
cv2.waitKey(0)

indices = np.where(skel==255)
im_verbose[indices[0], indices[1], :] = [0, 255, 0]

ransac = SIMPLE_CIRCLE_RANSAC( x_data=indices[1], y_data=indices[0], n=50000 )
ransac.execute_ransac()
a, b, r = ransac.best_model[0], ransac.best_model[1], ransac.best_model[2]
cv2.circle(im_verbose, (int(a),int(b)), int(r), (0,0,255),4)

cv2.imshow("ransac", im_verbose)


cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
"""
Ball detection by large kernel adaptiveTreshold -> findContours -> fitEllipse
Kwown ball radius used for false positives check
"""

import cv2
import numpy as np;
from random import randrange
from math import pi, fabs, sin, cos
from numpy.linalg import inv

#target ball radius
radius = 50

# Preparing input image
im_rgb = cv2.imread("../data/images/ball.png", cv2.IMREAD_COLOR)
im_rgb = cv2.pyrDown(im_rgb)
im_gray = cv2.cvtColor(im_rgb, cv2.COLOR_BGR2GRAY)

# Doing large kernel adaptive treshold
im = cv2.adaptiveThreshold(im_gray, 128, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,  cv2.THRESH_BINARY_INV, 101, -70)
cv2.imshow("adaptiveThreshold", im)
cv2.waitKey(0)

im_contours = cv2.cvtColor(im, cv2.COLOR_GRAY2BGR)
im_verbose = im_rgb.copy()

# Doing findCountours
contours, _ = cv2.findContours(im, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(im_contours, contours, -1, (0,0,255), 1)
cv2.imshow("drawContours", im_contours)
cv2.waitKey(0)

# Fitting ellipses for contours and checking their sizes. Correct ellipses will be drawn by red color
for contour in contours:
    if len(contour)>5:
        el = cv2.fitEllipse(contour)    
        color = (randrange(128,255), randrange(128,255), 0)
        w = el[1][0];
        h = el[1][1]
        if w > radius-10 and h > radius-10 and w < radius+10 and h < radius+10:
            color = (0,0,255)
        cv2.ellipse(im_verbose, el, color, 2)                
cv2.imshow("ellipses", im_verbose)


cv2.waitKey(0)
cv2.destroyAllWindows()

In [1]:
"""
Ball detection by GreenFilter -> DistanceTransform -> ConnectedComponents
Green filter used.
Target ball radius needed.
"""

import cv2
import numpy as np;
from random import randrange
from math import pi, fabs, sin, cos
from numpy.linalg import inv

im_rgb = cv2.imread("../data/images/ball.png", cv2.IMREAD_COLOR)
im_gray = cv2.cvtColor(im_rgb, cv2.COLOR_BGR2GRAY)

#target ball radius
radius = 50

# Simple green filter
im_hsv = cv2.cvtColor(im_rgb, cv2.COLOR_BGR2HSV)
im = cv2.cvtColor(im_hsv, cv2.COLOR_BGR2HSV)
hsv_min = np.array((20, 10, 35), np.uint8)
hsv_max = np.array((60, 125, 190), np.uint8)
im = cv2.inRange(im_hsv, hsv_min, hsv_max)
cv2.imshow("Green", im)
cv2.waitKey(0)

# Filtering speckle noise (false single pixels)
if False:
    # Doing heavy morphology filtering to remove speckle noise from 
    kernel = np.ones((3,3), np.uint8)
    im = cv2.dilate(im, kernel, iterations=1)
    im = cv2.erode(im, kernel, iterations=1)
    im = cv2.erode(im, kernel, iterations=1)
    im = cv2.dilate(im, kernel, iterations=1)
    im = cv2.dilate(im, kernel, iterations=1)
    im = cv2.dilate(im, kernel, iterations=1)
    im = cv2.erode(im, kernel, iterations=1)
else:
    im = cv2.medianBlur(im, 5)
    green_verbose = im_rgb.copy()
    im_not = cv2.bitwise_not(im)
    indices = np.where(im==255)
    
cv2.imshow("Green after speckle removal", im)
cv2.waitKey(0)

# Doing distance transform
dist_transform = cv2.distanceTransform(im_not, cv2.DIST_L2, 5) 
cv2.imshow("distanceTransform", dist_transform/100)
cv2.waitKey(0)

dist_candidates = cv2.cvtColor(dist_transform/100, cv2.COLOR_GRAY2BGR) # dist_candidates will be floating-point image here with pixel intensity [0..1]

#thresholding DistanceTransform by radius
ret, candidates = cv2.threshold(dist_transform, radius*0.8, 255,0)
candidates = np.uint8(candidates)
indices_c = np.where(candidates==255)
dist_candidates[indices_c[0], indices_c[1], :] = [0, 0, 1]
cv2.imshow("Candidates", dist_candidates)
cv2.waitKey(0)

# Extracting candidates centers
count, markers = cv2.connectedComponents(candidates)

# Checking each candidate pixel cluster
for i in range(count) :
    blob = np.where(markers==i)
    blob_area = len(blob[0])
    allowed_blob_radius = radius*0.2
    allowed_blob_area = pi * allowed_blob_radius * allowed_blob_radius

    blob_mask = np.zeros( (dist_transform.shape[0], dist_transform.shape[1], 1), np.uint8)
    blob_mask[blob[0], blob[1]] = 255
    
    if blob_area < allowed_blob_area:
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(dist_transform, mask=blob_mask)
        cv2.circle(im_rgb,max_loc,int(max_val),(0,0,255),5)

# Drawing the result
im_rgb[indices[0], indices[1], :] = [0, 255, 0]
cv2.imshow("Result", im_rgb)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
"""
Ball detection by OpenCV built-in SimpleBlobDetector after green filter
"""

import cv2
import numpy as np;


# Preparing input image image
im_rgb = cv2.imread("../data/images/ball.png", cv2.IMREAD_COLOR)


# Simple green filter
im_hsv = cv2.cvtColor(im_rgb, cv2.COLOR_BGR2HSV)
im = cv2.cvtColor(im_hsv, cv2.COLOR_BGR2HSV)
hsv_min = np.array((20, 10, 35), np.uint8)
hsv_max = np.array((60, 125, 190), np.uint8)
im = cv2.inRange(im_hsv, hsv_min, hsv_max)
cv2.imshow("Green", im)
cv2.waitKey(0)

# Morphology to remove gaps
kernel = np.ones((3,3), np.uint8)
im = cv2.dilate(im, kernel, iterations=1)
im = cv2.erode(im, kernel, iterations=1)
im = cv2.erode(im, kernel, iterations=1)
im = cv2.dilate(im, kernel, iterations=1)
cv2.imshow("Green morphology", im)
cv2.waitKey(0)


# Setup SimpleBlobDetector parameters.
params = cv2.SimpleBlobDetector_Params()

# Change thresholds
params.minThreshold = 10;
params.maxThreshold = 500;

# Filter by Area.
params.filterByArea = True
params.minArea = 1000
params.maxArea = 10000

# Filter by Circularity
params.filterByCircularity = True
params.minCircularity = 0.1

# Filter by Convexity
params.filterByConvexity = True
params.minConvexity = 0.87

# Filter by Inertia
params.filterByInertia = True
params.minInertiaRatio = 0.01

# Create a detector with the parameters
ver = (cv2.__version__).split('.')
if int(ver[0]) < 3 :
    detector = cv2.SimpleBlobDetector(params)
else :
    detector = cv2.SimpleBlobDetector_create(params)

# Detect blobs.
keypoints = detector.detect(im)

# Draw detected blobs as red circles.
# cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS ensures
# the size of the circle corresponds to the size of blob
im_with_keypoints = cv2.drawKeypoints(im, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

# Show blobs
cv2.imshow("Keypoints", im_with_keypoints)
cv2.waitKey(0)
cv2.destroyAllWindows()

# HSV vs RGB

In [1]:
import cv2
import time
import os
import math
import sys
import matplotlib 
matplotlib.use("Agg")
from IPython.display import clear_output

sys.path.append("../modules/")

import input_output
import processor


det = processor.Processors()
det.add_processor("rgb")
det.add_processor("hsv")

#gamma correction (lighting conditions change simulation) and online tuning
det.add_filter(processor.gamma_correction(), "rgb", "gamma correction")
det.add_filter(processor.crop(200, 300, 200, 300), "rgb", "crop")
det.add_filter(processor.calc_distribution(), "rgb", "hist")

det.add_filter(processor.gamma_correction(), "hsv", "gamma correction")
det.add_filter(processor.colorspace_to_colorspace("RGB", "HSV"), "hsv", "rgb2hsv")
det.add_filter(processor.calc_distribution(), "hsv", "hist")


cv2.namedWindow ('hsv')
cv2.createTrackbar ("gamma", "hsv", 100, 200, 
    lambda new_coeff : det.processors ["hsv"] ["gamma correction"].set_gamma (float (new_coeff) / 100))

cv2.namedWindow ('rgb')
cv2.createTrackbar ("gamma", "rgb", 100, 200, 
    lambda new_coeff : det.processors ["rgb"] ["gamma correction"].set_gamma (float (new_coeff) / 100))



source = input_output.Source ("../data/images/rgb_basket.jpg")
#source  = input_output.Source ("../data/output.avi")
#source  = input_output.Source ("-1")



while (True):
    frame = source.get_frame ()

    det.process(frame, "rgb")
    det.process(frame, "hsv")

    rgb_stages = det.get_stages_picts("rgb")
    hsv_stages = det.get_stages_picts("hsv")

    cv2.imshow ("rgb", input_output.form_grid(rgb_stages, window_x_sz=1000))
    cv2.imshow ("hsv", input_output.form_grid(hsv_stages[1:], window_x_sz=1000))

    time.sleep (0.02)
    keyb = cv2.waitKey (1) & 0xFF

    if (keyb == ord('q')):
        break
    
source.release()
cv2.destroyAllWindows()

arr shape (480, 640, 3)
arr shape (480, 640, 3)
before resize (482, 644, 3)
before resize (482, 644, 3)
before resize (482, 644, 3)
before resize (480, 640, 3)
before resize (482, 644, 3)
before resize (482, 644, 3)
before resize (480, 640, 3)
arr shape (480, 640, 3)
arr shape (480, 640, 3)
before resize (482, 644, 3)
before resize (482, 644, 3)
before resize (482, 644, 3)
before resize (480, 640, 3)
before resize (482, 644, 3)
before resize (482, 644, 3)
before resize (480, 640, 3)
arr shape (480, 640, 3)
arr shape (480, 640, 3)
before resize (482, 644, 3)
before resize (482, 644, 3)
before resize (482, 644, 3)
before resize (480, 640, 3)
before resize (482, 644, 3)
before resize (482, 644, 3)
before resize (480, 640, 3)
arr shape (480, 640, 3)
arr shape (480, 640, 3)
before resize (482, 644, 3)
before resize (482, 644, 3)
before resize (482, 644, 3)
before resize (480, 640, 3)
before resize (482, 644, 3)
before resize (482, 644, 3)
before resize (480, 640, 3)
arr shape (480, 640, 3)
